In [105]:
#bsub -nnodes 4 -Ip -XF -W 600 /bin/bash
!bjobs -X

/bin/bash: bjobs: command not found


In [106]:
allocated_hosts = ['lassen739', 'lassen583', 'lassen587', 'lassen588']
allocated_hosts_str = " ".join(allocated_hosts)
print(allocated_hosts_str)

lassen739 lassen583 lassen587 lassen588


In [107]:
from dask.distributed import SSHCluster
cluster = SSHCluster(
    allocated_hosts,
    connect_options={"known_hosts": None},
    worker_options={"nthreads": 40, 'nprocs':1},
    scheduler_options={"port": 8780, "dashboard_address": ":8797"}  
)

distributed.deploy.ssh - INFO - distributed.scheduler - INFO - -----------------------------------------------
distributed.deploy.ssh - INFO - distributed.http.proxy - INFO - To route to workers diagnostics web server please install jupyter-server-proxy: python -m pip install jupyter-server-proxy
distributed.deploy.ssh - INFO - distributed.scheduler - INFO - -----------------------------------------------
distributed.deploy.ssh - INFO - distributed.scheduler - INFO - Clear task state
distributed.deploy.ssh - INFO - distributed.scheduler - INFO -   Scheduler at: tcp://192.168.130.231:8780
distributed.deploy.ssh - INFO - distributed.nanny - INFO -         Start Nanny at: 'tcp://192.168.130.79:36373'
distributed.deploy.ssh - INFO - distributed.nanny - INFO -         Start Nanny at: 'tcp://192.168.130.80:43383'
distributed.deploy.ssh - INFO - distributed.nanny - INFO -         Start Nanny at: 'tcp://192.168.130.75:43431'
distributed.deploy.ssh - INFO - distributed.diskutils - INFO - Found 

In [108]:
#cluster.close()

In [54]:
import struct
import math
import pyarrow as pa
import pyarrow.parquet as pq
import sys, os
from pathlib import Path
import dask.dataframe as dd
from time import sleep
import pandas as pd
pd.options.display.max_columns = 2000
pd.set_option('display.width', 10000)
pd.set_option('display.max_colwidth', 10000)
from tqdm.notebook import tqdm, trange
from dask.distributed import Client, LocalCluster
from bokeh.plotting import figure, output_file, show
from bokeh.embed import components
from bokeh.models import FixedTicker, ColumnDataSource, LabelSet
from bokeh.io import output_notebook
output_notebook()
import numpy as np

import concurrent.futures
from concurrent.futures import ThreadPoolExecutor
import json
import gzip

Loading BokehJS ...

In [2]:
cluster = LocalCluster()

In [3]:
client = Client(cluster)
#client = Client("{}:8780".format(allocated_hosts[0]))

In [4]:
client

Connection method: Cluster object,Cluster type: LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 16
Total threads: 128,Total memory: 570.33 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:40199,Workers: 16
Dashboard: http://127.0.0.1:8787/status,Total threads: 128
Started: Just now,Total memory: 570.33 GiB
Comm: tcp://127.0.0.1:42509,Total threads: 8
Dashboard: http://127.0.0.1:41317/status,Memory: 35.65 GiB
Nanny: tcp://127.0.0.1:44799,


In [5]:
#client.shutdown()
#cluster.close()

In [6]:
parquet_folder="/p/gpfs1/haridev/recorder_logs/lbann_jag_32/_parquet"

In [56]:
chrome_timeline_folder = "/p/gpfs1/haridev/recorder_logs/lbann_jag_32/_chrome"
chrome_timeline_filename = "lbann_jag_32"
os.makedirs(chrome_timeline_folder, exist_ok=True)

In [7]:
ddf = dd.read_parquet("{}/*.parquet".format(parquet_folder), engine="pyarrow")

In [8]:
num_ranks = ddf['rank'].nunique().compute()
print(num_ranks)

128


In [9]:
ddf = ddf.sort_values('tstart')

In [10]:
num_rows = ddf.shape[0].compute()
print(num_rows)

79709114


In [11]:
ddf['index'] = ddf.assign(partition_count=1).partition_count.cumsum()

In [12]:
ddf = ddf.set_index('index', sorted=True)

In [13]:
posix_open = ["creat",        "creat64",      "open",         "open64"]
posix_read = ["read", "pread", "pread64", "readv"]
posix_write = ["write", "pwrite", "pwrite64","writev"]
posix_close = ["close"]
posix_calls = [         "lseek",        "lseek64", 
    "mmap",         "mmap64", "__xstat",      "__xstat64",    "__lxstat", "__lxstat64",
    "__fxstat",     "__fxstat64",   "getcwd",       "mkdir",    "rmdir",
    "chdir",        "link",         "linkat",       "unlink",   "symlink",
    "symlinkat",    "readlink",     "readlinkat",   "rename",   "chmod",
    "chown",        "lchown",       "utime",        "opendir",  "readdir",
    "closedir",     "rewinddir",    "mknod",        "mknodat",  "fcntl",
    "dup",          "dup2",         "pipe",         "mkfifo",   "umask",
    "fdopen",       "fileno",       "access",       "faccessat","tmpfile",
    "remove",       "truncate",     "ftruncate",    "vfprintf", "msync" ]

stdio_open = ["fopen", "fopen64"]
stdio_read = ["fread"]
stdio_write = ["fwrite"]
stdio_close = ["fclose"]
stdio_calls = ["ftell",        "fseek",    "fsync",
    "fdatasync", "fseeko",       "ftello"]

MPI_open = ["PMPI_File_open"]
MPI_read = ["PMPI_File_iread_at", "PMPI_File_iread", "PMPI_File_iread_shared", "PMPI_File_read_all_begin",     "PMPI_File_read_all",       "PMPI_File_read_at_all",
    "PMPI_File_read_at_all_begin",  "PMPI_File_read_at",        "PMPI_File_read",
    "PMPI_File_read_ordered_begin", "PMPI_File_read_ordered",   "PMPI_File_read_shared"]
MPI_write = ["PMPI_File_iwrite_at", "PMPI_File_iwrite",             "PMPI_File_iwrite_shared","PMPI_File_write_all_begin",
    "PMPI_File_write_all",          "PMPI_File_write_at_all_begin", "PMPI_File_write_at_all",
    "PMPI_File_write_at",           "PMPI_File_write",          "PMPI_File_write_ordered_begin",
    "PMPI_File_write_ordered",      "PMPI_File_write_shared"]
MPI_close = ["PMPI_File_close"]
HDF5_open = ["H5Fcreate",            "H5Fopen"]
HDF5_read = ["H5Dread"]
HDF5_write = ["H5Dwrite"]
HDF5_close = ["H5Fclose"]

In [14]:
io_funcs = posix_open
io_funcs.extend(posix_read)
io_funcs.extend(posix_write)
io_funcs.extend(posix_close)
io_funcs.extend(posix_calls)
io_funcs.extend(stdio_open)
io_funcs.extend(stdio_read)
io_funcs.extend(stdio_write)
io_funcs.extend(stdio_close)
io_funcs.extend(stdio_calls)
io_funcs.extend(MPI_open)
io_funcs.extend(MPI_read)
io_funcs.extend(MPI_write)
io_funcs.extend(MPI_close)
io_funcs.extend(HDF5_open)
io_funcs.extend(HDF5_read)
io_funcs.extend(HDF5_write)
io_funcs.extend(HDF5_close)

In [15]:
open_funcs = posix_open
open_funcs.extend(stdio_open)
open_funcs.extend(HDF5_open)
print(open_funcs)

close_funcs = posix_close
close_funcs.extend(stdio_close)
close_funcs.extend(HDF5_close)
print(close_funcs)

['creat', 'creat64', 'open', 'open64', 'read', 'pread', 'pread64', 'readv', 'write', 'pwrite', 'pwrite64', 'writev', 'close', 'lseek', 'lseek64', 'mmap', 'mmap64', '__xstat', '__xstat64', '__lxstat', '__lxstat64', '__fxstat', '__fxstat64', 'getcwd', 'mkdir', 'rmdir', 'chdir', 'link', 'linkat', 'unlink', 'symlink', 'symlinkat', 'readlink', 'readlinkat', 'rename', 'chmod', 'chown', 'lchown', 'utime', 'opendir', 'readdir', 'closedir', 'rewinddir', 'mknod', 'mknodat', 'fcntl', 'dup', 'dup2', 'pipe', 'mkfifo', 'umask', 'fdopen', 'fileno', 'access', 'faccessat', 'tmpfile', 'remove', 'truncate', 'ftruncate', 'vfprintf', 'msync', 'fopen', 'fopen64', 'fread', 'fwrite', 'fclose', 'ftell', 'fseek', 'fsync', 'fdatasync', 'fseeko', 'ftello', 'PMPI_File_open', 'PMPI_File_iread_at', 'PMPI_File_iread', 'PMPI_File_iread_shared', 'PMPI_File_read_all_begin', 'PMPI_File_read_all', 'PMPI_File_read_at_all', 'PMPI_File_read_at_all_begin', 'PMPI_File_read_at', 'PMPI_File_read', 'PMPI_File_read_ordered_begin',

In [16]:
relevant_filenames = ddf[(ddf["args_1"].str.contains("/p/gpfs1/"))]['args_1'].unique().compute()
print(relevant_filenames)

0    /p/gpfs1/haridev/software/lbann/applications/physics/ICF/20210824_103242_jag_wae/experiment.prototext
1                                                      /p/gpfs1/brainusr/datasets/jag/jag100K1vw_train.npy
Name: args_1, dtype: object


In [17]:
relevant_fds = ddf[(ddf["args_1"].isin(relevant_filenames))]['res'].unique().compute()
relevant_fds = relevant_fds.astype(str)
print(relevant_fds)

0    81
1    84
2    82
3    83
4    80
5    85
6    78
Name: res, dtype: object


In [18]:
open_rows = ddf[(ddf['func_id'].isin(open_funcs) & ddf['args_1'].isin(relevant_filenames)) |
                       (ddf['func_id'].isin(MPI_open) & ddf['args_2'].isin(relevant_filenames))].compute()
print(open_rows)

         rank     tstart       tend func_id  res  arg_count                                                                                                 args_1 args_2 args_3 args_4 args_5 args_6 args_7 args_8 args_9 args_10
index                                                                                                                                                                                                                             
1257070    33  20.513145  20.515337  open64   81          2  /p/gpfs1/haridev/software/lbann/applications/physics/ICF/20210824_103242_jag_wae/experiment.prototext      0                                                         
1257072    32  20.513147  20.515354  open64   84          2  /p/gpfs1/haridev/software/lbann/applications/physics/ICF/20210824_103242_jag_wae/experiment.prototext      0                                                         
1257099    34  20.513186  20.515369  open64   82          2  /p/gpfs1/haridev/software/lbann

In [19]:
close_rows = ddf[ddf['func_id'].isin(close_funcs) & ddf['args_1'].isin(relevant_fds)].compute()
print(close_rows)

          rank      tstart        tend func_id  res  arg_count args_1 args_2 args_3 args_4 args_5 args_6 args_7 args_8 args_9 args_10
index                                                                                                                                
1172074      0   20.067362   20.067387  fclose    0          1     78                                                                
1172098      0   20.067476   20.067499  fclose    0          1     78                                                                
1172116      0   20.067610   20.067631  fclose    0          1     78                                                                
1172129      0   20.067692   20.067713  fclose    0          1     78                                                                
1172140      0   20.067772   20.067795  fclose    0          1     78                                                                
...        ...         ...         ...     ...  ...        ...

In [20]:
an_array = np.empty(num_rows, dtype=str)
an_array[:] = np.nan
ddf['filename'] = dd.from_array(an_array)

In [21]:
io_ops = [dict() for x in range(num_ranks)]

In [22]:
print(io_funcs)

['creat', 'creat64', 'open', 'open64', 'read', 'pread', 'pread64', 'readv', 'write', 'pwrite', 'pwrite64', 'writev', 'close', 'lseek', 'lseek64', 'mmap', 'mmap64', '__xstat', '__xstat64', '__lxstat', '__lxstat64', '__fxstat', '__fxstat64', 'getcwd', 'mkdir', 'rmdir', 'chdir', 'link', 'linkat', 'unlink', 'symlink', 'symlinkat', 'readlink', 'readlinkat', 'rename', 'chmod', 'chown', 'lchown', 'utime', 'opendir', 'readdir', 'closedir', 'rewinddir', 'mknod', 'mknodat', 'fcntl', 'dup', 'dup2', 'pipe', 'mkfifo', 'umask', 'fdopen', 'fileno', 'access', 'faccessat', 'tmpfile', 'remove', 'truncate', 'ftruncate', 'vfprintf', 'msync', 'fopen', 'fopen64', 'fread', 'fwrite', 'fclose', 'ftell', 'fseek', 'fsync', 'fdatasync', 'fseeko', 'ftello', 'PMPI_File_open', 'PMPI_File_iread_at', 'PMPI_File_iread', 'PMPI_File_iread_shared', 'PMPI_File_read_all_begin', 'PMPI_File_read_all', 'PMPI_File_read_at_all', 'PMPI_File_read_at_all_begin', 'PMPI_File_read_at', 'PMPI_File_read', 'PMPI_File_read_ordered_begin',

In [23]:
for rank in range(num_ranks):
    ddf_rank = ddf[ddf['rank'] == rank]
    open_rows_rank = open_rows[open_rows['rank'] == rank]
    close_rows_rank = close_rows[close_rows['rank'] == rank]
    for filename in relevant_filenames:
        open_rows_file = open_rows_rank[open_rows_rank['args_1'].str.contains(filename) | 
                                        open_rows_rank['args_2'].str.contains(filename)]
        for i, open_row_file in open_rows_file.iterrows():
            #print(open_row_file)
            close_row_file = None
            val = str(open_row_file['func_id'])
            if val in ["open", "open64"]:
                fd = str(open_row_file['res'])
                close_row_file = close_rows_rank[(close_rows_rank.index > i) & 
                                            (close_rows_rank['func_id'] == "close") &
                                            (close_rows_rank['args_1'] == fd)].iloc[0]
            elif val == "fopen":
                fd = str(open_row_file['res'])
                close_row_file = close_rows_rank[(close_rows_rank.index > i) & 
                                            (close_rows_rank['func_id'] == "fclose") &
                                            (close_rows_rank['args_1'] == fd)].iloc[0]
            elif val == "PMPI_File_open":
                fd = str(open_row_file['res'])
                close_row_file = close_rows_rank[(close_rows_rank.index > i) & 
                                            (close_rows_rank['func_id'] == "PMPI_File_close") &
                                            (close_rows_rank['args_1'] == fd)].iloc[0]
            if close_row_file is not None:
                start_index = i
                end_index = int(close_row_file.name)
                indices = range(start_index, end_index + 1)
                print(rank, filename, indices[0], indices[-1])
                selected_records_file = ddf_rank.loc[start_index:indices[-1]]
                #selected_records_file['filename'] = filename
                io_ops[rank][filename] = {}
                io_ops[rank][filename]['df'] = selected_records_file
                #print(io_ops[rank])
            

0 /p/gpfs1/haridev/software/lbann/applications/physics/ICF/20210824_103242_jag_wae/experiment.prototext 1258106 1263607
0 /p/gpfs1/brainusr/datasets/jag/jag100K1vw_train.npy 2262231 2264442
1 /p/gpfs1/haridev/software/lbann/applications/physics/ICF/20210824_103242_jag_wae/experiment.prototext 1258429 1263646
1 /p/gpfs1/brainusr/datasets/jag/jag100K1vw_train.npy 2262235 2262590
2 /p/gpfs1/haridev/software/lbann/applications/physics/ICF/20210824_103242_jag_wae/experiment.prototext 1257973 1263662
2 /p/gpfs1/brainusr/datasets/jag/jag100K1vw_train.npy 2262240 2264443
3 /p/gpfs1/haridev/software/lbann/applications/physics/ICF/20210824_103242_jag_wae/experiment.prototext 1258068 1263593
3 /p/gpfs1/brainusr/datasets/jag/jag100K1vw_train.npy 2262241 2264444
4 /p/gpfs1/haridev/software/lbann/applications/physics/ICF/20210824_103242_jag_wae/experiment.prototext 1258159 1262769
4 /p/gpfs1/brainusr/datasets/jag/jag100K1vw_train.npy 2262402 2262714
5 /p/gpfs1/haridev/software/lbann/applications/phy

43 /p/gpfs1/haridev/software/lbann/applications/physics/ICF/20210824_103242_jag_wae/experiment.prototext 1257574 1262845
43 /p/gpfs1/brainusr/datasets/jag/jag100K1vw_train.npy 2262413 2265586
44 /p/gpfs1/haridev/software/lbann/applications/physics/ICF/20210824_103242_jag_wae/experiment.prototext 1257873 1262869
44 /p/gpfs1/brainusr/datasets/jag/jag100K1vw_train.npy 2262268 2262669
45 /p/gpfs1/haridev/software/lbann/applications/physics/ICF/20210824_103242_jag_wae/experiment.prototext 1257992 1262872
45 /p/gpfs1/brainusr/datasets/jag/jag100K1vw_train.npy 2262270 2262668
46 /p/gpfs1/haridev/software/lbann/applications/physics/ICF/20210824_103242_jag_wae/experiment.prototext 1258096 1262879
46 /p/gpfs1/brainusr/datasets/jag/jag100K1vw_train.npy 2262267 2265151
47 /p/gpfs1/haridev/software/lbann/applications/physics/ICF/20210824_103242_jag_wae/experiment.prototext 1258465 1262880
47 /p/gpfs1/brainusr/datasets/jag/jag100K1vw_train.npy 2262264 2265154
48 /p/gpfs1/haridev/software/lbann/appli

86 /p/gpfs1/haridev/software/lbann/applications/physics/ICF/20210824_103242_jag_wae/experiment.prototext 1258257 1258853
86 /p/gpfs1/brainusr/datasets/jag/jag100K1vw_train.npy 2262266 2265159
87 /p/gpfs1/haridev/software/lbann/applications/physics/ICF/20210824_103242_jag_wae/experiment.prototext 1258560 1258854
87 /p/gpfs1/brainusr/datasets/jag/jag100K1vw_train.npy 2262265 2265158
88 /p/gpfs1/haridev/software/lbann/applications/physics/ICF/20210824_103242_jag_wae/experiment.prototext 1258481 1263550
88 /p/gpfs1/brainusr/datasets/jag/jag100K1vw_train.npy 2262319 2264773
89 /p/gpfs1/haridev/software/lbann/applications/physics/ICF/20210824_103242_jag_wae/experiment.prototext 1258483 1263551
89 /p/gpfs1/brainusr/datasets/jag/jag100K1vw_train.npy 2262315 2264772
90 /p/gpfs1/haridev/software/lbann/applications/physics/ICF/20210824_103242_jag_wae/experiment.prototext 1258491 1263477
90 /p/gpfs1/brainusr/datasets/jag/jag100K1vw_train.npy 2262317 2263476
91 /p/gpfs1/haridev/software/lbann/appli

In [24]:
def thread_print(string):
    print(f'{string}\n', end='')
def compute_df(rank):
    for filename in relevant_filenames:
        #print(io_ops[rank])
        if filename in io_ops[rank]:
            io_ops[rank][filename]['size'] = Path(filename).stat().st_size
            io_ops[rank][filename]['df'] = io_ops[rank][filename]['df'].compute()
    
with ThreadPoolExecutor(max_workers = 40) as executor:
    future_gen = {executor.submit(compute_df, rank): rank for rank in range(num_ranks)}
    for future in concurrent.futures.as_completed(future_gen):
        rank = future_gen[future]
        try:
            data = future.result()
        except Exception as exc:
            thread_print('%r generated an exception: %s' % (rank, exc))
        else:
            thread_print('%r data computed' % (rank))

Task exception was never retrieved
future: <Task finished coro=<TCP.read() done, defined at /g/g92/haridev/.conda/envs/jupyter/lib/python3.7/site-packages/distributed/comm/tcp.py:189> exception=InvalidStateError('invalid state')>
Traceback (most recent call last):
  File "/g/g92/haridev/.conda/envs/jupyter/lib/python3.7/site-packages/distributed/comm/tcp.py", line 198, in read
    frames_nbytes = await stream.read_bytes(fmt_size)
concurrent.futures._base.CancelledError

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/g/g92/haridev/.conda/envs/jupyter/lib/python3.7/site-packages/distributed/comm/tcp.py", line 220, in read
    self.abort()
  File "/g/g92/haridev/.conda/envs/jupyter/lib/python3.7/site-packages/distributed/comm/tcp.py", line 326, in abort
    stream.close()
  File "/g/g92/haridev/.conda/envs/jupyter/lib/python3.7/site-packages/tornado/iostream.py", line 636, in close
    self._maybe_run_close_callback()
  Fil

13 data computed
30 data computed
9 data computed
15 data computed
19 data computed
0 data computed
28 data computed
6 data computed
32 data computed
29 data computed
2 data computed
7 data computed
16 data computed
4 data computed
24 data computed
22 data computed
39 data computed
37 data computed
8 data computed
17 data computed
18 data computed
36 data computed
5 data computed
11 data computed
27 data computed
1 data computed
20 data computed
10 data computed
34 data computed
31 data computed
14 data computed
23 data computed
25 data computed
21 data computed
33 data computed
35 data computed
38 data computed
12 data computed
26 data computed
3 data computed
67 data computed
64 data computed
48 data computed
68 data computed
73 data computed
76 data computed
74 data computed
51 data computed
49 data computed
75 data computed
72 data computed
77 data computed
62 data computed
63 data computed
69 data computed
42 data computed
45 data computed
60 data computed
58 data computed
44 data

In [25]:
print(io_ops[0][relevant_filenames[1]])

{'df':          rank     tstart       tend func_id  res  arg_count                                               args_1 args_2       args_3 args_4 args_5 args_6 args_7 args_8 args_9 args_10 filename
index                                                                                                                                                                                          
2262231     0  20.737125  20.737143   fopen   83          2  /p/gpfs1/brainusr/datasets/jag/jag100K1vw_train.npy     rb                                                                     NaN
2262244     0  20.737194  20.796473   fread    0          4                                                   %p      1           11     83                                                 NaN
2262511     0  21.974136  28.470678   fread    0          4                                                   %p      1  13405611264     83                                                 NaN
2264442     0  28.470713  28.4707

In [26]:
closeBook = {}  # Keep track the most recent close function and its file size so a later append operation knows the most recent file size
segmentBook = {}    # segmentBook[filename] maintains all segments for filename, it is a list of list (rank, segment-id, closed)
offsetBook = {}
endOfFile = {}  # endOfFile[filename][rank] keep tracks the end of file, only the local rank can see it. When close/fsync, the value is stored in closeBook so other rank can see it.
intervals = {}

# Initialize offset book, each rank maintains its own offset book
fdSet = set([0, 1, 2])  # stdin, stderr, stdout
segmentBook["stdin"] = []
segmentBook["stderr"] = []
segmentBook["stdout"] = []
endOfFile["stdin"] = [0] * num_ranks
endOfFile["stderr"] = [0] * num_ranks
endOfFile["stdout"] = [0] * num_ranks
for res in relevant_fds:
    fdSet.add(int(res))
for fd in fdSet:
    offsetBook[int(fd)] = [0] * num_ranks
for filename in relevant_filenames:
    segmentBook[filename] = []
    endOfFile[filename] = [0] * num_ranks
fileMaps = []
for i in range(num_ranks):
    fileMaps.append({0: "stdin", 1: "stdout", 2: "stderr"})

In [44]:
def handle_data_operations(record, fileMap, offsetBook, endOfFile):

    def update_end_of_file(rank, fd, filename, endOfFile, offsetBook):
        if filename in endOfFile and fd in offsetBook:
            endOfFile[filename][rank] = max(endOfFile[filename][rank], offsetBook[fd][rank])
        elif filename not in endOfFile:
            endOfFile[filename] = [0] * len(offsetBook[0])
            endOfFile[filename][rank] = offsetBook[fd][rank]
        else:
            print("Not possible: ", rank, fd, filename)
            endOfFile[filename][rank] = 0

    func = record['func_id']
    rank = record['rank']

    filename, offset, count = "", -1, -1

    # Ignore the functions that may confuse later conditions test
    if "readlink" in func or "dir" in func:
        return filename, offset, count

    if "writev" in func or "readv" in func:
        fd, count = int(record['args_1']), int(record['args_2'])
        if(fd not in fileMap): return "", -1, -1
        filename = fileMap[fd]

        offset = offsetBook[fd][rank]
        offsetBook[fd][rank] += count
        update_end_of_file(rank, fd, filename, endOfFile, offsetBook)
    elif "fwrite" in func or "fread" in func:
        fd, size, count = int(record['args_4']), int(record['args_2']), int(record['args_3'])
        #print(fileMap)
        if(fd not in fileMap): return "", -1, -1
        filename = fileMap[fd]

        offset, count = offsetBook[fd][rank], size*count
        offsetBook[fd][rank] += count
        update_end_of_file(rank, fd, filename, endOfFile, offsetBook)
    elif "pwrite" in func or "pread" in func:
        fd, count, offset = int(record['args_1']), int(record['args_3']), int(record['args_4'])
       
        if(fd not in fileMap): return "", -1, -1
        filename = fileMap[fd]

        update_end_of_file(rank, fd, filename, endOfFile, offsetBook)
    elif "write" in func or "read" in func:
        fd, count = int(record['args_1']), int(record['args_3'])
        if(fd not in fileMap): return "", -1, -1
        filename = fileMap[fd]

        offset = offsetBook[fd][rank]
        offsetBook[fd][rank] += count
        update_end_of_file(rank, fd, filename, endOfFile, offsetBook)
    elif "fprintf" in func:
        fd, count = int(record['args_1']), int(record['args_2'])
        if(fd not in fileMap): return "", -1, -1
        filename = fileMap[fd]

        offset = offsetBook[fd][rank]
        offsetBook[fd][rank] += count
        update_end_of_file(rank, fd, filename, endOfFile, offsetBook)
    #print(filename, offset, count)
    return filename, offset, count


def handle_metadata_operations(record, fileMap, offsetBook, closeBook, segmentBook, endOfFile):

    def get_latest_offset(filename, rank, closeBook, endOfFile):
        # Every filename should be in endOfFile becuase we initialized it at the begining
        if filename not in closeBook and filename not in endOfFile:
            print("Encounter an unknown filename "+filename+". Should abort!")
            return 0
        if filename in closeBook:
            return max(endOfFile[filename][rank], closeBook[filename])
        else:
            return endOfFile[filename][rank]

    def create_new_segment(filename, rank, segmentBook):
        newSegmentID = 0
        if filename in segmentBook and len(segmentBook[filename]) > 0:
            newSegmentID = 1+segmentBook[filename][-1][1]
        if filename not in segmentBook:
            segmentBook[filename] = []
        segmentBook[filename].append([rank, newSegmentID, False])

    rank, func = record['rank'], record['func_id']

    # Ignore directory related operations
    if "dir" in func:
        return

    if "fopen" in func or "fdopen" in func:
        fd = record['res']
        if "fdopen" in func:
            oldFd = int(record['args_1'])
            if oldFd not in fileMap:        # openning some file that we do not know?
                return
            else:
                filename = fileMap[oldFd]
        else:
            filename = record['args_1']
        fileMap[fd] = filename
        offsetBook[fd][rank] = 0
        openMode = record['args_2']
        if 'a' in openMode:
            offsetBook[fd][rank] = get_latest_offset(filename, rank, closeBook, endOfFile)
        create_new_segment(filename, rank, segmentBook)
    elif "open" in func:
        fd = record['res']
        filename = record['args_1']
        fileMap[fd] = filename
        offsetBook[fd][rank] = 0
        openMode = int( record['args_2'] )
        if openMode == 2:  # TODO need  a better way to test for O_APPEND
            offsetBook[fd][rank] = get_latest_offset(filename, rank, closeBook, endOfFile)
        create_new_segment(filename, rank, segmentBook)
    elif "seek" in func:
        fd, offset, whence = int(record['args_1']), int(record['args_2']), int(record['args_3'])
        if fd not in fileMap: return
        filename = fileMap[fd]

        if whence == 0:     # SEEK_SET
            offsetBook[fd][rank] = offset
        elif whence == 1:   # SEEK_CUR
            offsetBook[fd][rank] += offset
        elif whence == 2:   # SEEK_END
            offsetBook[fd][rank] = get_latest_offset(filename, rank, closeBook, endOfFile)

    elif "close" in func or "sync" in func:
        fd = int(record['args_1'])
        if(fd not in fileMap): return
        filename = fileMap[fd]
        if "close" in func: del fileMap[fd]

        closeBook[filename] = endOfFile[filename][rank]

        # 1. Close all segments on the local process for this file
        for i in range(len(segmentBook[filename])):
            if segmentBook[filename][i][0] == rank:
                segmentBook[filename][i][2] = True
        # 2. And starts a new segment for all other processes have the same file opened
        # Skip this step for session semantics
        visitedRanks = set()
        tmpSegments = []
        # [::-1] check the most recent unclosed segment to get the largest segmentId
        for segment in segmentBook[filename][::-1]:
            if segment[0] in visitedRanks:
                continue
            if segment[0] != rank and not segment[2]:
                tmpSegments.append([segment[0], 1+segment[1], False])
                visitedRanks.add(segment[0])
        segmentBook[filename] = segmentBook[filename] + tmpSegments


def ignore_files(filename):
    if not filename or filename == "":
        return True
    ignore_prefixes = ["/sys/", "/dev", "/proc", "/etc/", "stdout", "stderr", "stdin"]
    for prefix in ignore_prefixes:
        if filename.startswith(prefix):
            return True
    if "pipe:" in filename:
        return True

    return False

def ignore_funcs(func):
    ignore = ["MPI", "H5", "writev"]
    for f in ignore:
        if f in func:
            return True
    return False

In [45]:
for rank in range(num_ranks):
    iops = io_ops[rank]
    for filename in iops:
        interval = {}        
        for index, record in iops[filename]['df'].iterrows():
            index_map = {'offset':0, 'count':0}
            fileMap = fileMaps[rank]
            func = record['func_id']
            #print(f'Handle Meta on {index}', end='\r')
            handle_metadata_operations(record, fileMap, offsetBook, closeBook, segmentBook, endOfFile)

            #print(f'Handle Data on {index}', end='\r')
            fname, offset, count = handle_data_operations(record, fileMap, offsetBook, endOfFile)
            index_map['offset'] = offset
            index_map['count'] = count
            interval[str(index)] = index_map
        iops[filename]['piece'] = interval

In [46]:
print(io_ops[0][relevant_filenames[1]])

{'df':          rank     tstart       tend func_id  res  arg_count                                               args_1 args_2       args_3 args_4 args_5 args_6 args_7 args_8 args_9 args_10 filename
index                                                                                                                                                                                          
2262231     0  20.737125  20.737143   fopen   83          2  /p/gpfs1/brainusr/datasets/jag/jag100K1vw_train.npy     rb                                                                     NaN
2262244     0  20.737194  20.796473   fread    0          4                                                   %p      1           11     83                                                 NaN
2262511     0  21.974136  28.470678   fread    0          4                                                   %p      1  13405611264     83                                                 NaN
2264442     0  28.470713  28.4707

In [47]:
chromeTimeline = {
    "traceEvents": [],
    "displayTimeUnit": "ms",
    "systemTraceEvents": "SystemTraceData",
    "otherData": {
        "version": "VaniDL v1.0"
    },
    "stackFrames": {},
    "samples": []
}

In [49]:
timeshift = 0
data = []

In [52]:
for rank in range(num_ranks):
    iops = io_ops[rank]
    for filename in iops:
        open_rec = iops[filename]['df'].iloc[0]
        category = "NA"
        if open_rec['func_id'] in posix_open:
            category = "POSIX"
        elif open_rec['func_id'] in stdio_open:
            category = "STDIO"
        elif open_rec['func_id'] in MPI_open:
            category = "MPI-IO"
        elif open_rec['func_id'] in HDF5_open:
            category = "HDF5"
        for index, row in iops[filename]['df'].iterrows():
            interval = iops[filename]['piece'][str(index)]
            event_start = {"name": filename, "cat": category, "ph": "B",
                           "ts": int(float(row['tstart'] + float(timeshift)) * 1e6), "pid": rank, "tid": 0,
                           "args": {
                               "Module": category,
                               "Filename": filename,
                               "Rank": rank,
                               "Operation": row['func_id'],
                               "Offset": interval['offset'],
                               "Length": interval['count'],
                               "Start": row['tstart'],
                               "End": row['tend']
                           }
            }
            event_end = {"ph": "E", "ts": int((float(row['tend']) + float(timeshift)) * 1e6), "pid": rank, "tid": 0,
                         "args": {
                               "Module": category,
                               "Filename": filename,
                               "Rank": rank,
                               "Operation": row['func_id'],
                               "Offset": interval['offset'],
                               "Length": interval['count'],
                               "Start": row['tstart'],
                               "End": row['tend']
                         }
            }
            data.append(event_start)
            data.append(event_end)
        

In [57]:
chromeTimeline["traceEvents"] = data

json_str = json.dumps(chromeTimeline) + "\n"
json_bytes = json_str.encode('utf-8')
with gzip.GzipFile("{}/{}.gz".format(chrome_timeline_folder, chrome_timeline_filename), 'w') as fout:  # 4. gzip
    fout.write(json_bytes)

In [58]:
file = "{}/{}.gz".format(chrome_timeline_folder, chrome_timeline_filename)
!ls -l $file

-rw------- 1 haridev haridev 37189 Sep  6 11:27 /p/gpfs1/haridev/recorder_logs/lbann_jag_32/_chrome/lbann_jag_32.gz


In [17]:
relevant_fds = ddf[(ddf["args_1"].isin(relevant_filenames))]['res'].unique().compute()
relevant_fds = relevant_fds.astype(str)
print(relevant_fds)

0    81
1    84
2    82
3    83
4    80
5    85
6    78
Name: res, dtype: object


In [18]:
reduced_ddf = ddf[~(ddf['func_id'].isin(open_funcs) & ~ddf['args_1'].isin(relevant_filenames))]

In [19]:
reduced_ddf = reduced_ddf[~(reduced_ddf['func_id'].isin(MPI_open) & ~reduced_ddf['args_2'].isin(relevant_filenames))]

In [20]:
reduced_ddf = reduced_ddf[~(reduced_ddf['func_id'].isin(posix_read) & ~reduced_ddf['args_1'].isin(relevant_fds))]
reduced_ddf = reduced_ddf[~(reduced_ddf['func_id'].isin(posix_write) & ~reduced_ddf['args_1'].isin(relevant_fds))]
reduced_ddf = reduced_ddf[~(reduced_ddf['func_id'].str.contains("close") & ~reduced_ddf['args_1'].isin(relevant_fds))]

In [21]:
reduced_ddf = reduced_ddf[~(reduced_ddf['func_id'].isin(stdio_read) & ~reduced_ddf['args_4'].isin(relevant_fds))]
reduced_ddf = reduced_ddf[~(reduced_ddf['func_id'].isin(stdio_write) & ~reduced_ddf['args_4'].isin(relevant_fds))]
reduced_ddf = reduced_ddf[~(reduced_ddf['func_id'].str.contains("fclose") & ~reduced_ddf['args_1'].isin(relevant_fds))]

In [22]:
relevant_io_funcs = ["writev" , "readv","fwrite" , 
                     "fread","pwrite" , "pread", "write" , "read",
                    "fprintf", "fopen" , "fdopen", "open", "open64", "seek", "fseek", "close", "sync", "fsync","lseek",        "lseek64",  "pread",
    "pread64",      "pwrite",       "pwrite64",     "readv",    "writev","fopen",        "fopen64",  "fclose",
    "fwrite",       "fread",        "ftell",        "fseek",    "fsync",
    "fdatasync"]

In [23]:
reduced_ddf = reduced_ddf[reduced_ddf['func_id'].isin(relevant_io_funcs)]

In [24]:
num_rows = len(reduced_ddf)
print(num_rows)

482977


In [25]:
filenames = reduced_ddf[(reduced_ddf["func_id"].isin(open_funcs))]['args_1'].unique().compute()
print(filenames)

0    /p/gpfs1/haridev/software/lbann/applications/physics/ICF/20210824_103242_jag_wae/experiment.prototext
1                                                      /p/gpfs1/brainusr/datasets/jag/jag100K1vw_train.npy
Name: args_1, dtype: object


In [26]:
reduced_ddf = reduced_ddf[~reduced_ddf['func_id'].str.contains("MPI")]

In [27]:
reduced_ddf = reduced_ddf.compute()

In [28]:
num_rows = reduced_ddf.shape[0]
print(num_rows)

482977


In [29]:
reduced_ddf = reduced_ddf.sort_values('tstart')

In [30]:
reduced_ddf[reduced_ddf['func_id'].str.contains("read")]

,rank,tstart,tend,func_id,res,arg_count,args_1,args_2,args_3,args_4,args_5,args_6,args_7,args_8,args_9,args_10
9391,0,20.067341,20.067358,fread,0,4,%p,1,255,78,,,,,,
9394,0,20.067459,20.067472,fread,0,4,%p,1,255,78,,,,,,
9397,0,20.067593,20.067604,fread,0,4,%p,1,255,78,,,,,,
9400,0,20.067675,20.067686,fread,0,4,%p,1,255,78,,,,,,
9403,0,20.067755,20.067768,fread,0,4,%p,1,255,78,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17721,125,30.126385,30.126387,read,0,3,84,%p,8,,,,,,,
17725,125,30.126537,30.126537,read,0,3,84,%p,8,,,,,,,
17728,125,30.126570,30.126570,read,0,3,84,%p,8,,,,,,,
17819,1,30.153996,30.153999,read,0,3,84,%p,8,,,,,,,


In [31]:
reduced_ddf

,rank,tstart,tend,func_id,res,arg_count,args_1,args_2,args_3,args_4,args_5,args_6,args_7,args_8,args_9,args_10
33,2,2.364273,2.364276,fdopen,52,2,52,r,,,,,,,,
31,3,2.364273,2.364276,fdopen,52,2,52,r,,,,,,,,
33,0,2.364284,2.364288,fdopen,53,2,53,r,,,,,,,,
33,1,2.364294,2.364297,fdopen,52,2,52,r,,,,,,,,
32,4,2.364381,2.364383,fdopen,52,2,52,r,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
622435,31,146.374847,146.374847,close,0,1,83,,,,,,,,,
622436,31,146.374863,146.374878,close,0,1,84,,,,,,,,,
622437,31,146.374878,146.374908,close,0,1,85,,,,,,,,,
624902,2,146.380478,146.380493,close,0,1,85,,,,,,,,,


In [32]:
#ddf = None
#client.shutdown()
#cluster.close()

In [33]:
relevant_filenames = reduced_ddf[(reduced_ddf["args_1"].str.contains("/p/gpfs1/"))]['args_1'].unique()
print(relevant_filenames)

['/p/gpfs1/haridev/software/lbann/applications/physics/ICF/20210824_103242_jag_wae/experiment.prototext'
 '/p/gpfs1/brainusr/datasets/jag/jag100K1vw_train.npy']


In [34]:
closeBook = {}  # Keep track the most recent close function and its file size so a later append operation knows the most recent file size
segmentBook = {}    # segmentBook[filename] maintains all segments for filename, it is a list of list (rank, segment-id, closed)
offsetBook = {}
endOfFile = {}  # endOfFile[filename][rank] keep tracks the end of file, only the local rank can see it. When close/fsync, the value is stored in closeBook so other rank can see it.
intervals = {}

# Initialize offset book, each rank maintains its own offset book
fdSet = set([0, 1, 2])  # stdin, stderr, stdout
segmentBook["stdin"] = []
segmentBook["stderr"] = []
segmentBook["stdout"] = []
endOfFile["stdin"] = [0] * num_ranks
endOfFile["stderr"] = [0] * num_ranks
endOfFile["stdout"] = [0] * num_ranks
for res in relevant_fds:
    fdSet.add(res)
for fd in fdSet:
    offsetBook[int(fd)] = [0] * num_ranks
for filename in relevant_filenames:
    segmentBook[filename] = []
    endOfFile[filename] = [0] * num_ranks
fileMaps = []
for i in range(num_ranks):
    fileMaps.append({0: "stdin", 1: "stdout", 2: "stderr"})

In [39]:
def handle_data_operations(record, fileMap, offsetBook, endOfFile):

    def update_end_of_file(rank, fd, filename, endOfFile, offsetBook):
        if filename in endOfFile and fd in offsetBook:
            endOfFile[filename][rank] = max(endOfFile[filename][rank], offsetBook[fd][rank])
        elif filename not in endOfFile:
            endOfFile[filename] = [0] * len(offsetBook[0])
            endOfFile[filename][rank] = offsetBook[fd][rank]
        else:
            print("Not possible: ", rank, fd, filename)
            endOfFile[filename][rank] = 0

    func = record['func_id']
    rank = record['rank']

    filename, offset, count = "", -1, -1

    # Ignore the functions that may confuse later conditions test
    if "readlink" in func or "dir" in func:
        return filename, offset, count

    if "writev" in func or "readv" in func:
        fd, count = int(record['args_1']), int(record['args_2'])
        if(fd not in fileMap): return "", -1, -1
        filename = fileMap[fd]

        offset = offsetBook[fd][rank]
        offsetBook[fd][rank] += count
        update_end_of_file(rank, fd, filename, endOfFile, offsetBook)
    elif "fwrite" in func or "fread" in func:
        fd, size, count = int(record['args_4']), int(record['args_2']), int(record['args_3'])
        if(fd not in fileMap): return "", -1, -1
        filename = fileMap[fd]

        offset, count = offsetBook[fd][rank], size*count
        offsetBook[fd][rank] += count
        update_end_of_file(rank, fd, filename, endOfFile, offsetBook)
    elif "pwrite" in func or "pread" in func:
        fd, count, offset = int(record['args_1']), int(record['args_3']), int(record['args_4'])
        if(fd not in fileMap): return "", -1, -1
        filename = fileMap[fd]

        update_end_of_file(rank, fd, filename, endOfFile, offsetBook)
    elif "write" in func or "read" in func:
        fd, count = int(record['args_1']), int(record['args_3'])
        if(fd not in fileMap): return "", -1, -1
        filename = fileMap[fd]

        offset = offsetBook[fd][rank]
        offsetBook[fd][rank] += count
        update_end_of_file(rank, fd, filename, endOfFile, offsetBook)
    elif "fprintf" in func:
        fd, count = int(record['args_1']), int(record['args_2'])
        if(fd not in fileMap): return "", -1, -1
        filename = fileMap[fd]

        offset = offsetBook[fd][rank]
        offsetBook[fd][rank] += count
        update_end_of_file(rank, fd, filename, endOfFile, offsetBook)

    return filename, offset, count


def handle_metadata_operations(record, fileMap, offsetBook, closeBook, segmentBook, endOfFile):

    def get_latest_offset(filename, rank, closeBook, endOfFile):
        # Every filename should be in endOfFile becuase we initialized it at the begining
        if filename not in closeBook and filename not in endOfFile:
            print("Encounter an unknown filename "+filename+". Should abort!")
            return 0
        if filename in closeBook:
            return max(endOfFile[filename][rank], closeBook[filename])
        else:
            return endOfFile[filename][rank]

    def create_new_segment(filename, rank, segmentBook):
        newSegmentID = 0
        if filename in segmentBook and len(segmentBook[filename]) > 0:
            newSegmentID = 1+segmentBook[filename][-1][1]
        if filename not in segmentBook:
            segmentBook[filename] = []
        segmentBook[filename].append([rank, newSegmentID, False])

    rank, func = record['rank'], record['func_id']

    # Ignore directory related operations
    if "dir" in func:
        return

    if "fopen" in func or "fdopen" in func:
        fd = record['res']
        if "fdopen" in func:
            oldFd = int(record['args_1'])
            if oldFd not in fileMap:        # openning some file that we do not know?
                return
            else:
                filename = fileMap[oldFd]
        else:
            filename = record['args_1']
        fileMap[fd] = filename
        offsetBook[fd][rank] = 0
        openMode = record['args_2']
        if 'a' in openMode:
            offsetBook[fd][rank] = get_latest_offset(filename, rank, closeBook, endOfFile)
        create_new_segment(filename, rank, segmentBook)
    elif "open" in func:
        fd = record['res']
        filename = record['args_1']
        fileMap[fd] = filename
        offsetBook[fd][rank] = 0
        openMode = int( record['args_2'] )
        if openMode == 2:  # TODO need  a better way to test for O_APPEND
            offsetBook[fd][rank] = get_latest_offset(filename, rank, closeBook, endOfFile)
        create_new_segment(filename, rank, segmentBook)
    elif "seek" in func:
        fd, offset, whence = int(record['args_1']), int(record['args_2']), int(record['args_3'])
        if fd not in fileMap: return
        filename = fileMap[fd]

        if whence == 0:     # SEEK_SET
            offsetBook[fd][rank] = offset
        elif whence == 1:   # SEEK_CUR
            offsetBook[fd][rank] += offset
        elif whence == 2:   # SEEK_END
            offsetBook[fd][rank] = get_latest_offset(filename, rank, closeBook, endOfFile)

    elif "close" in func or "sync" in func:
        fd = int(record['args_1'])
        if(fd not in fileMap): return
        filename = fileMap[fd]
        if "close" in func: del fileMap[fd]

        closeBook[filename] = endOfFile[filename][rank]

        # 1. Close all segments on the local process for this file
        for i in range(len(segmentBook[filename])):
            if segmentBook[filename][i][0] == rank:
                segmentBook[filename][i][2] = True
        # 2. And starts a new segment for all other processes have the same file opened
        # Skip this step for session semantics
        visitedRanks = set()
        tmpSegments = []
        # [::-1] check the most recent unclosed segment to get the largest segmentId
        for segment in segmentBook[filename][::-1]:
            if segment[0] in visitedRanks:
                continue
            if segment[0] != rank and not segment[2]:
                tmpSegments.append([segment[0], 1+segment[1], False])
                visitedRanks.add(segment[0])
        segmentBook[filename] = segmentBook[filename] + tmpSegments


def ignore_files(filename):
    if not filename or filename == "":
        return True
    ignore_prefixes = ["/sys/", "/dev", "/proc", "/etc/", "stdout", "stderr", "stdin"]
    for prefix in ignore_prefixes:
        if filename.startswith(prefix):
            return True
    if "pipe:" in filename:
        return True

    return False

def ignore_funcs(func):
    ignore = ["MPI", "H5", "writev"]
    for f in ignore:
        if f in func:
            return True
    return False

In [40]:
#bar = trange(num_rows)
for index, record in tqdm(reduced_ddf.iterrows(), total=num_rows):
    #bar.set_description(f'Working on "{index}"')
    #print(f'Working on {index}', end='\r')
    
    rank = record['rank']
    #print(rank)
    fileMap = fileMaps[rank]

    func = record['func_id']

    #print(f'Handle Meta on {index}', end='\r')
    handle_metadata_operations(record, fileMap, offsetBook, closeBook, segmentBook, endOfFile)
    
    #print(f'Handle Data on {index}', end='\r')
    filename, offset, count = handle_data_operations(record, fileMap, offsetBook, endOfFile)
    
    #if filename:
    #    print(filename, offset, count)
    if not ignore_files(filename):
        isRead = "read" in func
        if filename not in intervals:
            intervals[filename] = []

        # segments[0] is the local segment, the others are remote segments
        segments = []
        # 1. Add local segment
        # Find the most recent unclosed local segment
        #print(f'Local Segment on {index}', end='\r')
        for segment in segmentBook[filename][::-1]:
            if segment[0] == rank and not segment[2]:
                segments.append(segment[1])

        # 2. Add all remote segmentsv
        #print(f'Remote Segment on {index}', end='\r')
        for segment in segmentBook[filename]:
            if segment[0] != rank and not segment[2]:
                segments.append(segment[1])
        intervals[filename].append( [rank, record.tstart, record.tend, offset, count, isRead, segments] )

  0%|          | 0/482977 [00:00<?, ?it/s]

In [41]:
def function_patterns(all_intervals):
    # 1,2,3 - consecutive
    # 1,3,9 - sequential
    # 1,3,2 - random
    x = {'consecutive':0, 'sequential':0, 'random':0}
    for filename in all_intervals.keys():
        if ignore_files(filename): continue
        intervals = sorted(all_intervals[filename], key=lambda x: x[1])   # sort by tstart
        '''
        This code consider each rank separately
        lastOffsets = [0] * reader.globalMetadata.numRanks
        for interval in intervals:
            rank, offset, count = interval[0], interval[3], interval[4]
            lastOffset = lastOffsets[rank]
            if (offset + count) == lastOffset:
                x['consecutive'] += 1
            elif (offset + count) > lastOffset:
                x['sequential'] += 1
            else:
                #print filename, interval
                x['random'] += 1
            lastOffsets[rank] = offset + count
        '''
        for i in range(len(intervals)-1):
            i1, i2 = intervals[i], intervals[i+1]
            offset1, count1 = i1[3], i1[4]
            offset2, count2 = i2[3], i2[4]

            if (offset1 + count1) == offset2:
                x['consecutive'] += 1
            elif (offset1 + count1) < offset2:
                x['sequential'] += 1
            else:
                x['random'] += 1
        total = x['consecutive'] + x['sequential'] + x['random']
    print("consecutive:",  x['consecutive'] )
    print("sequential:",  x['sequential'] )
    print("random:",  x['random'])

In [42]:
function_patterns(intervals)

consecutive: 5
sequential: 0
random: 633


In [43]:
def pattern_for_one_file(filename, intervals):
        pattern = {"RAR": {'S':0, 'D':0}, "RAW": {'S':0, 'D':0},
                    "WAW": {'S':0, 'D':0}, "WAR": {'S':0, 'D':0}}
        intervals = sorted(intervals, key=lambda x: x[3])   # sort by starting offset
        for i in range(len(intervals)-1):
            i1, i2 = intervals[i], intervals[i+1]
            tstart1, offset1, count1, segments1 = i1[1], i1[3], i1[4], i1[6]
            tstart2, offset2, count2, segments2 = i2[1], i2[3], i2[4], i2[6]

            # no overlapping
            if offset1+count1 <= offset2:
                continue
            if len(segments1) == 0 or len(segments2) ==0:
                #print("Without a session? ", filename, i1, i2)
                continue
            # has overlapping but may not conflicting
            # if segments1 intersets segments2, and
            # one of the common segments is the local session
            # then there's a conflict
            if not (segments1[0] in segments2 or segments2[0] in segments1):
                continue

            isRead1 = i1[5] if tstart1 < tstart2 else i2[5]
            isRead2 = i2[5] if tstart2 > tstart1 else i1[5]
            rank1 = i1[0] if tstart1 < tstart2 else i2[0]
            rank2 = i2[0] if tstart2 > tstart1 else i1[0]

            #print(filename, i1, i2)
            # overlap
            if isRead1 and isRead2:             # RAR
                if rank1 == rank2: pattern['RAR']['S'] += 1
                else: pattern['RAR']['D'] += 1
            if isRead1 and not isRead2:         # WAR
                if rank1 == rank2: pattern['WAR']['S'] += 1
                else: pattern['WAR']['D'] += 1
            if not isRead1 and not isRead2:     # WAW
                if rank1 == rank2: pattern['WAW']['S'] += 1
                else: pattern['WAW']['D'] += 1
            if not isRead1 and isRead2:         # RAW
                if rank1 == rank2: pattern['RAW']['S'] += 1
                else: pattern['RAW']['D'] += 1
        # debug info
        if pattern['RAW']['S']: print("RAW-S", pattern['RAW']['S'], filename)
        if pattern['RAW']['D']: print("RAW-D", pattern['RAW']['D'], filename)
        if pattern['WAW']['S']: print("WAW-S", pattern['WAW']['S'], filename)
        if pattern['WAW']['D']: print("WAW-D", pattern['WAW']['D'], filename)
        if pattern['WAR']['S']: print("WAR-S", pattern['WAR']['S'], filename)
        if pattern['WAR']['D']: print("WAR-D", pattern['WAR']['D'], filename)
        return pattern


In [44]:
for filename in relevant_filenames:
    print(pattern_for_one_file(filename, intervals[filename]))

{'RAR': {'S': 0, 'D': 381}, 'RAW': {'S': 0, 'D': 0}, 'WAW': {'S': 0, 'D': 0}, 'WAR': {'S': 0, 'D': 0}}
{'RAR': {'S': 0, 'D': 254}, 'RAW': {'S': 0, 'D': 0}, 'WAW': {'S': 0, 'D': 0}, 'WAR': {'S': 0, 'D': 0}}


In [45]:
def offset_vs_time(intervals):
    # interval = [rank, tstart, tend, offset, count]
    def plot_for_one_file(filename, intervals):
        intervals = sorted(intervals, key=lambda x: x[1])   # sort by tstart
        x_read, y_read, x_write, y_write, nan = [], [], [], [], float('nan')
        for interval in intervals:
            tstart, tend, offset, count, isRead = interval[1], interval[2], interval[3], interval[4], interval[5]
            if isRead:
                x_read += [tstart, tend, nan]
                y_read += [offset, offset+count, offset+count]
            else:
                x_write += [tstart, tend, nan]
                y_write += [offset, offset+count, offset+count]

        if len(x_read) > 0 : x_read = x_read[0:len(x_read)-1]
        if len(y_read) > 0 : y_read = y_read[0:len(y_read)-1]
        if len(x_write) > 0 : x_write = x_write[0:len(x_write)-1]
        if len(y_write) > 0 : y_write = y_write[0:len(y_write)-1]
        p = figure(title=filename.split("/")[-1], x_axis_label="Time", y_axis_label="Offset")
        p.line(x_read, y_read, line_color='blue', line_width=2, alpha=1.0, legend_label="read")
        p.line(x_write, y_write, line_color='red', line_width=2, alpha=1.0, legend_label="write")
        return p


    plots = []
    idx = 0
    for filename in intervals:
        if ignore_files(filename): continue
        if 'junk' in filename and int(filename.split('junk.')[-1]) > 0: continue    # NWChem
        if 'pout' in filename and int(filename.split('pout.')[-1]) > 0: continue    # Chombo
        if idx < 16 and (len(intervals[filename]) > 0): # only show 12 files at most
            p = plot_for_one_file(filename, intervals[filename])
            plots.append(p)
            show(p)
            idx += 1
            
    

In [46]:
offset_vs_time(intervals)

In [47]:
def io_sizes(intervals, read=True):

    sizes = {}
    for filename in intervals:
        if ignore_files(filename): continue
        for interval in intervals[filename]:
            io_size , isRead = interval[4], interval[5]
            if read != isRead: continue
            if io_size not in sizes: sizes[io_size] = 0
            sizes[io_size] += 1

    xs = sorted(sizes.keys())
    ys = [ sizes[x] for x in xs ]
    xs = [ str(x) for x in xs ]

    p = figure(x_range=xs, x_axis_label="IO Size", y_axis_label="Count", y_axis_type='log', plot_width=500 if not read else 400, plot_height=350)
    p.vbar(x=xs, top=ys, width=0.6, bottom=1)
    p.xaxis.major_label_orientation = math.pi/2

    labels = LabelSet(x='x', y='y', text='y', level='glyph', x_offset=-10, y_offset=0, text_font_size="10pt",
                source=ColumnDataSource(dict(x=xs ,y=ys)), render_mode='canvas')
    p.add_layout(labels)
    show(p)

In [48]:
io_sizes(intervals)

In [49]:
def offset_vs_rank(intervals):
    # interval = [rank, tstart, tend, offset, count]
    def plot_for_one_file(filename, intervals):
        intervals = sorted(intervals, key=lambda x: x[3])   # sort by starting offset
        x_read, y_read, x_write, y_write, nan = [], [], [], [], float('nan')
        for interval in intervals:
            rank, offset, count, isRead = interval[0], interval[3], interval[4], interval[5]
            if isRead:
                x_read += [rank, rank, rank]
                y_read += [offset, offset+count, nan]
            else:
                x_write += [rank, rank, rank]
                y_write += [offset, offset+count, nan]

        if len(x_read) > 0 : x_read = x_read[0:len(x_read)-1]
        if len(y_read) > 0 : y_read = y_read[0:len(y_read)-1]
        if len(x_write) > 0 : x_write = x_write[0:len(x_write)-1]
        if len(y_write) > 0 : y_write = y_write[0:len(y_write)-1]
        p = figure(title=filename.split("/")[-1], x_axis_label="Rank", y_axis_label="Offset")
        p.line(x_read, y_read, line_color='blue', line_width=5, alpha=1.0, legend_label="read")
        p.line(x_write, y_write, line_color='red', line_width=5, alpha=1.0, legend_label="write")
        return p

    plots = []
    idx = 0
    for filename in intervals:
        if ignore_files(filename): continue
        if 'junk' in filename and int(filename.split('junk.')[-1]) > 0: continue    # NWChem
        if 'pout' in filename and int(filename.split('pout.')[-1]) > 0: continue    # Chombo
        if idx < 16 and (len(intervals[filename]) > 0): # only show 12 files at most
            p = plot_for_one_file(filename, intervals[filename])
            plots.append(p)
            show(p)
            idx += 1


In [50]:
offset_vs_rank(intervals)

In [51]:
def overall_io_activities(df):

    nan = float('nan')

    def io_activity(rank):
        x_read, x_write, y_read, y_write = [], [], [], []
        rank_df = df[df['rank'] == rank]
        records_per_rank = rank_df.shape[0]
        for index, record in rank_df.iterrows():
            funcname = record['func_id']
            if "MPI" in funcname or "H5" in funcname: continue
            if "write" in funcname or "fprintf" in funcname:
                x_write.append(record['tstart'])
                x_write.append(record['tend'])
                x_write.append(nan)
            if "read" in funcname:
                x_read.append(record['tstart'])
                x_read.append(record['tend'])
                x_read.append(nan)

        if(len(x_write)>0): x_write = x_write[0: len(x_write)-1]
        if(len(x_read)>0): x_read = x_read[0: len(x_read)-1]

        y_write = [rank] * len(x_write)
        y_read = [rank] * len(x_read)

        return x_read, x_write, y_read, y_write


    p = figure(x_axis_label="Time", y_axis_label="Rank", plot_width=600, plot_height=400)
    for rank in tqdm(range(4)):
        x_read, x_write, y_read, y_write = io_activity(rank)
        p.line(x_write, y_write, line_color='red', line_width=20, alpha=1.0, legend_label="write")
        p.line(x_read, y_read, line_color='blue', line_width=20, alpha=1.0, legend_label="read")

    p.legend.location = "top_left"
    show(p)

In [52]:
overall_io_activities(reduced_ddf)

  0%|          | 0/4 [00:00<?, ?it/s]

In [53]:
def function_times(df):

    aggregate = {}
    for rank in tqdm(range(num_ranks)):
        rank_df = df[df['rank'] == rank]
        records_per_rank = rank_df.shape[0]
        for index, record in rank_df.iterrows():
            if record['func_id'] in aggregate:
                aggregate[record['func_id']] += (record['tend'] - record['tstart'])
            else:
                aggregate[record['func_id']] = (record['tend'] - record['tstart'])
                
                
    funcnames, times = np.array([]), np.array([])
    for key in aggregate:
        if aggregate[key] > 0:
            funcnames = np.append(funcnames, key)
            times = np.append(times, aggregate[key])

    index = np.argsort(times)[::-1]
    times = times[index]
    funcnames = funcnames[index]

    p = figure(x_axis_label="Spent Time (Seconds)", y_axis_label="Function", y_range=funcnames)
    p.hbar(y=funcnames, right=times, height=0.8, left=0)
    labels = LabelSet(x='x', y='y', text='x', level='glyph', x_offset=0, y_offset=-8, text_font_size="10pt",
                source=ColumnDataSource(dict(x=times, y=funcnames)), render_mode='canvas')
    p.add_layout(labels)
    show(p)


In [54]:
function_times(reduced_ddf)

  0%|          | 0/128 [00:00<?, ?it/s]

tornado.application - ERROR - Exception in callback functools.partial(<function wrap.<locals>.null_wrapper at 0x200141189290>, <Future finished exception=StreamClosedError('Stream is closed')>)
Traceback (most recent call last):
  File "/g/g92/haridev/.conda/envs/jupyter/lib/python3.7/site-packages/tornado/gen.py", line 1141, in run
    yielded = self.gen.throw(*exc_info)
  File "/g/g92/haridev/.conda/envs/jupyter/lib/python3.7/site-packages/tornado/tcpclient.py", line 232, in connect
    af, addr, stream = yield connector.start(connect_timeout=timeout)
  File "/g/g92/haridev/.conda/envs/jupyter/lib/python3.7/site-packages/tornado/gen.py", line 1133, in run
    value = future.result()
  File "/g/g92/haridev/.conda/envs/jupyter/lib/python3.7/site-packages/tornado/tcpclient.py", line 112, in on_connect_done
    stream = future.result()
tornado.iostream.StreamClosedError: Stream is closed

During handling of the above exception, another exception occurred:

Traceback (most recent call las